# This script processes the o3memo results

1. Compile o3memo (both memo policies), o3memoA (only memoA) and o3memoB (only memoB) with:
```
make bin/o3memo
make bin/o3memoA
make bin/o3memoB
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_optim3.py
```
3. The output that we will process is `o3_memoizationA_<hostname>.csv`, `o3_memoization_<hostname>.csv`, and `o3_memoizationB_<hostname>.csv`, that should be already saved in the `Results` folder.


In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the files exist
file_list = [                   
                f'o2_partree_{hostname}.csv',
                f'o3_memoizationA_{hostname}.csv',
                f'o3_memoization_{hostname}.csv',
                f'o3_memoizationB_{hostname}.csv',
            ]

for o3_file in file_list:
    # if it is not in Results
    if not os.path.exists(o3_file):
        # is the file already in the directory?
        assert os.path.exists(os.path.join(hostname, o3_file)), f'File {o3_file} not found: something went wrong with the baseline benchmark.'
    # if it is in Results
    else:
        # copy the file to the directory
        assert os.system(f'mv {o3_file} {hostname}/') == 0, f'Failed to move {o3_file} to {hostname}/'
# add the path to all the names in the list
for i in range(len(file_list)):
    file_list[i] = os.path.join(hostname, file_list[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean
from common.utils import get_nprocs, get_best_comb

cloud_list = ['AlcoyH','ArzuaH','BrionFH','BrionUH']
# number of threads used in the execution
num_threads = get_nprocs()

def tokenize(filename):
    results ={}
    with open(filename) as f:
        next(f) # skip header
        for line in f:
            tokens = line.split()
            name=tokens[1].split("/")[3][:-5]
            nth=int(tokens[2])
            level=int(tokens[5])
            if name not in results:
                results[name]={}
            if level not in results[name]:
                results[name][level]={}
            results[name][level][nth]=[float(tokens[6])] # tree construction time
            results[name][level][nth].append(float(tokens[7])) # owm time
            results[name][level][nth].append(float(tokens[6])+float(tokens[7])) # total time
            results[name][level][nth].append(float(tokens[8])) # accuracy
    return results

def tokenize2(filename):
    results ={}
    with open(filename) as f:
        next(f) # skip header
        for line in f:
            tokens = line.split()
            name=tokens[0].split("/")[3][:-5]
            nth=int(tokens[1])
            level=int(tokens[4])
            if name not in results:
                results[name]={}
            if level not in results[name]:
                results[name][level]={}
            results[name][level][nth]=[float(tokens[5])] # tree construction time
            results[name][level][nth].append(float(tokens[6])) # owm time
            results[name][level][nth].append(float(tokens[5])+float(tokens[6])) # total time
            results[name][level][nth].append(float(tokens[7])) # accuracy
    return results

reso2=tokenize(f"{hostname}/o2_partree_{hostname}.csv")
reso3A=tokenize(f"{hostname}/o3_memoizationA_{hostname}.csv") # optim3 without with just memo B
reso3=tokenize(f"{hostname}/o3_memoization_{hostname}.csv") # optim3 without with memo A and B
reso3B=tokenize(f"{hostname}/o3_memoizationB_{hostname}.csv") # optim3 without with just memo B


In [ ]:
def plot_levels(results, message, vnth):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(['Alcoy','Arzua','BrionF','BrionU']):
        _,nth,_ = get_best_comb(results[name], vnth)
        x=list(results[name].keys())
        axs[i].plot(np.array(x), np.array([results[name][j][nth][1] for j in x]), marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][j][nth][0] for j in x]), marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][j][nth][2] for j in x]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Level', fontsize=xlabelfs)
        axs[i].set_xticks(x)
        axs[i].tick_params(axis='x', labelsize=xticksfs)        
        axs[i].grid()
    if(nth==1):
        fig.suptitle(f'Execution time (sec.) @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    if(nth>1):
        fig.suptitle(f'{message} {nth}-threads execution time (sec.) @ {hostname.upper()}',  fontweight='bold', fontsize=18)

    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    # pp = PdfPages("Optim3_8coresExecTime.pdf")
    # pp.savefig(fig)
    # pp.close()

plot_levels(reso2, "Optim2", num_threads)
plot_levels(reso3A, "Optim3 (A)", num_threads)
plot_levels(reso3B, "Optim3 (B)", num_threads)
plot_levels(reso3, "Optim3 (A+B)", num_threads)


In [ ]:
def plot_times(phase, results, message, vnth):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['Tree Const.','OWM Trav.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(['Alcoy','Arzua','BrionF','BrionU']):
        for j in range(4):
            # bestlevel = min(results[j][name], key=lambda x: results[j][name][x][nth][2])
            bestlevel,_,_ = get_best_comb(results[j][name], vnth)
            x=list(results[j][name][bestlevel].keys())
            axs[i].plot(np.array(x), np.array([results[j][name][bestlevel][nth][phase] for nth in x]), marks[j], linewidth=linew, markersize=markers)
        
        axs[i].set_title(name+ " (lev="+str(bestlevel)+")",fontsize=16)
        axs[i].set_xlabel('Num threads', fontsize=xlabelfs)
        axs[i].set_xticks(x)
        axs[i].set_xticks(x)
        axs[i].tick_params(axis='x', labelsize=xticksfs)        
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()

    fig.suptitle(labels[phase]+' Execution time (sec.)',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(message,loc='best', fontsize= 14)
    #axs[i].show()

plot_times(0,[reso2,reso3A,reso3B,reso3],["Optim2","Optim3 (A)","Optim3 (B)","Optim3 (A+B)"], num_threads)
plot_times(1,[reso2,reso3A,reso3B,reso3],["Optim2","Optim3 (A)","Optim3 (B)","Optim3 (A+B)"], num_threads)
plot_times(2,[reso2,reso3A,reso3B,reso3],["Optim2","Optim3 (A)","Optim3 (B)","Optim3 (A+B)"], num_threads)

# Compare OWM traversal improvement

In [ ]:
def printall(phase,vnth,res,message):
    labels=['Tree Const.','OWM Trav.','Total']
    # get the clouds name
    clouds = res[0]
    # get the best conf (level,num procs,Total time) from o3 memo A+B
    bestconf = [get_best_comb(res[3][cloud], vnth) for cloud in clouds]

    for i,cloud in enumerate(clouds):
        print("Cloud {} has best level {} for Total time".format(cloud, bestconf[i][0]))

    # for j,n in zip(res[0],range(len(res[0]))):
    for j,cloud in enumerate(clouds):
        time=[] 
        for i in range(len(res)):
            time.append(res[i][cloud][bestconf[j][0]][bestconf[j][1]][phase])  
            print("Cloud {} has {}-cores {} {} time = {:.4f} (speedup vs Optim2 = {:.2f}x).".format(cloud, bestconf[j][1], labels[phase], message[i], time[i], time[0]/time[i]))

printall(1, num_threads, [reso2,reso3A,reso3B,reso3], ["Optim2", "Optim3 (A)", "Optim3 (B)", "Optim3 (A+B)"])

# Compare Tree construction improvement

In [ ]:
printall(0, num_threads,[reso2,reso3A,reso3B,reso3],["Optim2","Optim3 (A)","Optim3 (B)","Optim3 (A+B)"])

# Compare Total (OWM traversal + Tree construction) improvement

In [ ]:
printall(2, num_threads, [reso2,reso3A,reso3B,reso3], ["Optim2","Optim3 (A)","Optim3 (B)","Optim3 (A+B)"]) 

In [ ]:
#Configuration variables
def plot_res(res, vnth):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8

    marks=['o-','x-','s-','v-','+-']
    # get the best conf (level,num procs,Total time)
    bestconf = [get_best_comb(res[i], vnth) for i in res]
    # keep only the best level
    bestlevel = [i[0] for i in bestconf]

    labels=['Tree Const.','OWM Trav.','Total']

    fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(15, 5), constrained_layout=True)
    sizes=[i+" (lev="+str(j)+")" for i,j in zip(res,bestlevel)]
    sizes.append('Ideal')

    for subfig,phase in enumerate(['qtree','owm','total']):
        print("Phase: "+phase+ " speedup for each cloud and best level for each cloud: "+str(list(res.keys())))
        for (i,z,lev) in zip(res,marks,bestlevel):
            print(np.array([res[i][lev][1][subfig]/res[i][lev][j][subfig] for j in vnth]))
            axs[subfig].plot(np.array(vnth), np.array([res[i][lev][1][subfig]/res[i][lev][j][subfig] for j in vnth]), z, linewidth=linew, markersize=markers)

        axs[subfig].plot(np.array(vnth), np.array(vnth), '-', linewidth=linew, markersize=markers)
        axs[subfig].set_title('Speedup '+labels[subfig],  fontweight='bold', fontsize=titlefs)
        axs[subfig].set_xlabel('Number of cores', fontsize=xlabelfs)
        axs[subfig].set_xticks(vnth)
        axs[subfig].tick_params(axis='x', labelsize=xticksfs)
        axs[subfig].grid()

    axs[0].set_ylabel('Speedup', fontsize=ylabelfs)
    axs[0].legend(sizes,loc='best', fontsize= legendfs)
    
    # plt.show()
    return fig

fig = plot_res(reso3, num_threads)

In [ ]:
pp = PdfPages(os.path.join(hostname, f'Speedup-o3-memoization-{hostname}.pdf'))
pp.savefig(fig)
pp.close()

# Save results in All_Optimizations.csv

In [ ]:
output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")

for i in reso3A:
    bestlevel,nth,_ = get_best_comb(reso3A[i], num_threads)
    print("Opt3-MemoA; {}; {:.5f}; {:.5f};{};{}".format(i,reso3A[i][bestlevel][nth][0],reso3A[i][bestlevel][nth][1],bestlevel,0))
    f.write("Opt3-MemoA;{};{:.5f};{:.5f};{};{}\n".format(i,reso3A[i][bestlevel][nth][0],reso3A[i][bestlevel][nth][1],bestlevel,0))
for i in reso3B:
    bestlevel,nth,_ = get_best_comb(reso3B[i], num_threads)
    print("Opt3-MemoB; {}; {:.5f}; {:.5f};{};{}".format(i,reso3B[i][bestlevel][nth][0],reso3B[i][bestlevel][nth][1],bestlevel,0))
    f.write("Opt3-MemoB;{};{:.5f};{:.5f};{};{}\n".format(i,reso3B[i][bestlevel][nth][0],reso3B[i][bestlevel][nth][1],bestlevel,0))
for i in reso3:
    bestlevel,nth,_ = get_best_comb(reso3[i], num_threads)
    print("Opt3-Memo; {}; {:.5f}; {:.5f};{};{}".format(i,reso3[i][bestlevel][nth][0],reso3[i][bestlevel][nth][1],bestlevel,0))
    f.write("Opt3-Memo;{};{:.5f};{:.5f};{};{}\n".format(i,reso3[i][bestlevel][nth][0],reso3[i][bestlevel][nth][1],bestlevel,0))

f.close()
